## 410 Final Project: Generating Summaries for News Articles
Aaron Kuhstoss, Shalin Mehta, and Aleksandra Grigortsuk

### Imports

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from langdetect import detect
import random 

from rouge import Rouge
from bert_score import score

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

 ### Data Preprocessing
 

In [30]:
# Import the dataset
df = pd.read_csv("Latest_News.csv")
print(len(df))

# Filtering dataset with English articles and non-NA 
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

# Subset df to make it workable, since python is computationally slow
random.seed(410)
df_subset = df.sample(n=15000)

# Apply the language detection function to df
df_subset['detected_language'] = df_subset['content'].apply(detect_language)
english_articles = df_subset[df_subset['detected_language'] == 'en']

86560


In [33]:
# Print the total number of articles in the 'english_articles' DataFrame
print(len(english_articles))

# Calculate and print the average length of the 'content' field in the 'english_articles' DataFrame
print(english_articles['content'].str.len().mean())

# Calculate and print the average length of the 'description' field in the 'english_articles' DataFrame
print(english_articles['description'].str.len().mean())

# Count and print the number of missing (null) values in the 'content' field of the 'english_articles' DataFrame
print(english_articles['content'].isnull().sum())

# Count and print the number of missing (null) values in the 'title' field of the 'english_articles' DataFrame
print(english_articles['title'].isnull().sum())

# Display the first few rows of the 'english_articles' DataFrame to get a quick overview of the data
(english_articles.head())

1207
2268.2038111019056
249.67170818505338
0
0


,title,link,keywords,creator,video_url,description,content,pubDate,full_description,image_url,source_id,detected_language
28360,Where Inter And Juventus Are At After Derby D’...,https://www.forbes.com/sites/emmetgates/2021/1...,"['SportsMoney', '/sportsmoney', 'Business', '/...","['Emmet Gates', ' Contributor']",NaN,Both sides played out a cagey affair at San Si...,Both sides played out a cagey affair at San Si...,2021-10-25 14:51:31,Share to Facebook Share to Twitter Share to Li...,https://thumbor.forbes.com/thumbor/fit-in/0x0/...,forbes,en
51727,Jubilant Pharmova Q2 Review - Radiopharma Busi...,https://www.bloombergquint.com/research-report...,"['Jubilant Pharmova Ltd.', 'bqblue']",['Motilal Oswal Financial Services'],NaN,Jubilant Pharmova Q2 Review - Radiopharma Busi...,BQ Blue’s special research section collates qu...,2021-10-25 05:42:18,NaN,NaN,bloombergquint,en
63945,"Joanna Cameron, Star of DC's The Secrets of Is...",https://comicbook.com/dc/news/joanna-camerona-...,NaN,['Adam Barnhardt'],NaN,"Joanna Cameron, star of CBS' The Secrets of Is...","✖Joanna Cameron, star of CBS' The Secrets of I...",2021-10-24 20:50:00,"✖ Joanna Cameron, star of CBS' The Secrets of ...",https://sportshub.cbsistatic.com/i/2021/10/24/...,comicbook,en
81622,Parents more hesitant to vaccinate kids than t...,https://panow.com/2021/10/24/parents-more-hesi...,NaN,NaN,NaN,OTTAWA - Jennifer Hubert jumped at the opportu...,OTTAWA — Jennifer Hubert jumped at the opportu...,2021-10-24 11:02:48,OTTAWA — Jennifer Hubert jumped at the opportu...,https://s3.amazonaws.com/socast-superdesk/medi...,panow,en
30579,"Salting routes at full capacity, meeting hears",https://kildare-nationalist.ie/2021/10/25/salt...,['Kildare News'],['Conor Forrest'],NaN,Kildare Co Council has been asked to include t...,Kildare Co Council has been asked to include t...,2021-10-25 14:00:54,NaN,https://kildare-nationalist.ie/wp-content/uplo...,kildare-nationalist,en


### Pipeline Construction
##### Summarization pipeline

A detailed description of what the function does and how it achives its summaries is found in the comments:

In [36]:
def summarize(text, per):
    # Load the English language model from Spacy
    nlp = spacy.load('en_core_web_sm')

    # Process the input text and tokenize it
    doc = nlp(text)
    tokens = [token.text for token in doc]

    # Initialize a dictionary to hold word frequencies
    word_frequencies = {}

    # Calculate the frequency of each word in the text, excluding stopwords and punctuation
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1

    # Normalize word frequencies by dividing each by the maximum frequency
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency

    # Break the text into sentences
    sentence_tokens = [sent for sent in doc.sents]
    sentence_scores = {}

    # Score sentences based on the frequency of the words they contain
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent] = word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.text.lower()]

    # Determine the number of sentences to include in the summary
    select_length = int(len(sentence_tokens) * per)

    # Select the sentences with the highest scores
    summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)

    # Combine the selected sentences into a final summary
    final_summary = [word.text for word in summary]
    summary = ''.join(final_summary)

    # Return the summarized text
    return summary

### Creating Summaries

The first cell directly below prints out 10 summaries from the summaries list using our algorithm. It tends to be a little time consuming (taking about 10 nminutes), but does indeed create the summaries for each article in our pre-preprocessed data. 

The second cell selectively generates summaries and stores them along with the corresponding articles but only does so for 5 articles starting from the 500th article. This makes the generation time alot quicker, and is what we use in our model evaluation.

Both cells use a summary ratio of 0.1 for the creation of the summaries. 

In [41]:
# Initialize an empty list to store the generated summaries
summaries_1 = []

# Iterate over each article in the 'english_articles' DataFrame
for i in range(len(english_articles)):
    # Generate a summary of the article's content, summarized to 10% of its original length
    summary = summarize(english_articles.iloc[i,6], 0.1)

    # If a summary is successfully generated, add it to the summaries list
    if summary:
        summaries_1.append(summary)

# Retrieve and display the first 10 summaries from the generated list
summaries_1[:10]

KeyboardInterrupt: 

In [38]:
# Initialize lists to store generated summaries and corresponding raw articles
summaries_2 = []
raw_articles = []

# Start at the 500th article in the 'english_articles' DataFrame
index = 500

# Continue generating summaries until we have 5 of them
while len(summaries_2) < 5:
    # Retrieve the article at the current index
    article = english_articles.iloc[index,:]
    # Extract the content of the article
    article_content = english_articles.iloc[index,6]

    # Generate a summary of the article content, summarized to 10% of its original length
    summary = summarize(article_content, 0.1)

    # If a summary is successfully generated, add it to the summaries list and the corresponding article to the raw articles list
    if summary:
        summaries_2.append(summary)
        raw_articles.append(article)

    # Move to the next article
    index += 1

### Model Evaluation

In [39]:
"""
NOTE: A fundamental issue with using ROUGE and BLEU metrics is their dependence on high-quality reference summaries.
Using the "description" field as a pseudo summary provides a good amount of reference data. However, the quality of these descriptions varies, and some are null.
Another approach is to write summaries by hand for comparison, but this is time-consuming and limits the data points.
"""

# Obtain original descriptions for use as reference "summaries"
descriptions = [row.iloc[5] for row in raw_articles]

# Get raw content of articles for BERTscoring
orig_articles = [row.iloc[6] for row in raw_articles]

# Uncomment the below lines to display raw article content and descriptions
# print('Raw Articles:', '\n'.join(orig_articles))
# print('Descriptions:', '\n'.join(descriptions))

# Uncomment the below line to display summaries for qualitative evaluation
# print('Summaries:', '\n'.join(summaries))

# Hand-written summaries for evaluation
own_summaries = [
    "Bryan Cranstons portrayal of the morally gray character Walter White in the hit series Breaking Bad has been loved by fans. However, AMC had other choices for the lead, with Breaking Bad creator Vince Gilligan ultimately persuading executives to choose Cranston.",
    "Recently on TikTok, the term weaponized incompetence is gaining a lot of attention. According to psychotherapist and writer, Emily Mendez, M.S. EdS, “Weaponized incompetence refers to pretending not to know how to do something when you do really know how to do it.” The term has 21.8M views on TikTok as example, mostly of women, whose colleagues, partners, and family members use weaponized incompetence to get out of work.",
    "The All India Congress is going to launch a country wide protest from November 14 against the abnormal rise of fuel. The massive protest against the high fuel price will start from November 14 and will continue till November 29, after five consecutive days of rising fuel prices across the country.",
    "Numerous artists across multiple genres, such as Lil Nas X, Ariana Grande, and Olivia Rodrigo are entering songs for consideration in the upcoming Grammy award season. This includes Justin Bieber, whose smash hit “Peaches” (featuring Daniel Caesar and Giveon) is vying for a Grammy nomination as best R&B performance.",
    "Singer-songwriter Ed Sheeran announced Sunday he had tested positive for COVID-19 and would be self-isolating in his home five days before he is scheduled to release his fourth studio album. Sheeran's upcoming album, titled '=' is scheduled to be released on October 29."
]


##### To see the evaluation metrics we have for each of the 5 articles using BERT and ROUGE, please select the "scrollable element" option in the output below (because it is truncated by default)

##### We decided to compare our summaries using both metrics on our own hand-written samples as well as the description given in the dataset. 

In [40]:
# Function to evaluate summaries using Rouge and BERTscores
def eval_summaries(generated_summaries, hand_written_summaries, descriptions, content):
    """
    This function evaluates the generated summaries using ROUGE and BERTscores,
    comparing them with hand-written summaries and article descriptions.
    It prints the precision, recall, and F1 scores for each summary,
    indicating the type of reference used.
    """
    # Evaluate against hand-written summaries
    print("===== Evaluation Against Hand-Written Summaries =====")
    _evaluate_each_summary(generated_summaries, hand_written_summaries, content, "Hand-Written")

    # Evaluate against article descriptions
    print("\n===== Evaluation Against Article Descriptions =====")
    _evaluate_each_summary(generated_summaries, descriptions, content, "Article Description")

def _evaluate_each_summary(summaries, references, content, reference_type):
    # Initialize the ROUGE object
    rouge = Rouge()
    rouge_scores = rouge.get_scores(summaries, references)

    # Displaying ROUGE scores
    for index, score_set in enumerate(rouge_scores):
        print(f"--- Summary {index + 1} (Compared with {reference_type}) ---")
        print("ROUGE Scores:")
        for rouge_key, values in score_set.items():
            print(f"  {rouge_key.upper()}:")
            for metric, value in values.items():
                print(f"    {metric.capitalize()}: {value:.4f}")
        print()

    # Get BERTscore
    P, R, F1 = score(summaries, content, lang='en')

    # Display BERTscore results
    print("BERTscore Results:")
    for i in range(len(summaries)):
        print(f"Summary {i+1} (Compared with {reference_type}):")
        print(f"  Precision: {P[i].item():.4f}, Recall: {R[i].item():.4f}, F1: {F1[i].item():.4f}")
        print()

# Evaluating generated summaries
eval_summaries(summaries_2, own_summaries, descriptions, orig_articles)


===== Evaluation Against Hand-Written Summaries =====
--- Summary 1 (Compared with Hand-Written) ---
ROUGE Scores:
  ROUGE-1:
    R: 0.0541
    P: 0.1053
    F: 0.0714
  ROUGE-2:
    R: 0.0000
    P: 0.0000
    F: 0.0000
  ROUGE-L:
    R: 0.0541
    P: 0.1053
    F: 0.0714

--- Summary 2 (Compared with Hand-Written) ---
ROUGE Scores:
  ROUGE-1:
    R: 0.0909
    P: 0.1250
    F: 0.1053
  ROUGE-2:
    R: 0.0000
    P: 0.0000
    F: 0.0000
  ROUGE-L:
    R: 0.0909
    P: 0.1250
    F: 0.1053

--- Summary 3 (Compared with Hand-Written) ---
ROUGE Scores:
  ROUGE-1:
    R: 0.1622
    P: 0.0938
    F: 0.1188
  ROUGE-2:
    R: 0.0000
    P: 0.0000
    F: 0.0000
  ROUGE-L:
    R: 0.1351
    P: 0.0781
    F: 0.0990

--- Summary 4 (Compared with Hand-Written) ---
ROUGE Scores:
  ROUGE-1:
    R: 0.1333
    P: 0.0938
    F: 0.1101
  ROUGE-2:
    R: 0.0000
    P: 0.0000
    F: 0.0000
  ROUGE-L:
    R: 0.1111
    P: 0.0781
    F: 0.0917

--- Summary 5 (Compared with Hand-Written) ---
ROUGE Scores:
 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTscore Results:
Summary 1 (Compared with Hand-Written):
  Precision: 0.9336, Recall: 0.8083, F1: 0.8665

Summary 2 (Compared with Hand-Written):
  Precision: 0.9676, Recall: 0.8141, F1: 0.8842

Summary 3 (Compared with Hand-Written):
  Precision: 0.8631, Recall: 0.7871, F1: 0.8233

Summary 4 (Compared with Hand-Written):
  Precision: 0.9586, Recall: 0.8104, F1: 0.8783

Summary 5 (Compared with Hand-Written):
  Precision: 0.9295, Recall: 0.7889, F1: 0.8534


===== Evaluation Against Article Descriptions =====
--- Summary 1 (Compared with Article Description) ---
ROUGE Scores:
  ROUGE-1:
    R: 0.2857
    P: 0.2105
    F: 0.2424
  ROUGE-2:
    R: 0.0000
    P: 0.0000
    F: 0.0000
  ROUGE-L:
    R: 0.2143
    P: 0.1579
    F: 0.1818

--- Summary 2 (Compared with Article Description) ---
ROUGE Scores:
  ROUGE-1:
    R: 0.8298
    P: 0.9750
    F: 0.8966
  ROUGE-2:
    R: 0.8302
    P: 0.9778
    F: 0.8980
  ROUGE-L:
    R: 0.8298
    P: 0.9750
    F: 0.8966

--- Summary 3 (Compared wit

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTscore Results:
Summary 1 (Compared with Article Description):
  Precision: 0.9336, Recall: 0.8083, F1: 0.8665

Summary 2 (Compared with Article Description):
  Precision: 0.9676, Recall: 0.8141, F1: 0.8842

Summary 3 (Compared with Article Description):
  Precision: 0.8631, Recall: 0.7871, F1: 0.8233

Summary 4 (Compared with Article Description):
  Precision: 0.9586, Recall: 0.8104, F1: 0.8783

Summary 5 (Compared with Article Description):
  Precision: 0.9295, Recall: 0.7889, F1: 0.8534

